<a href="https://colab.research.google.com/github/entenam/TF-Intro/blob/main/HorseHumanCNNClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Path to GDrive Folder
path = '/content/drive/MyDrive/AI Basics with Tensorflow/HorseHumanCNNClassification/'


In [3]:
# Download Horse/Human Training Dataset to GDrive Designated Folder
!gdown -O /content/drive/MyDrive/AI\ Basics\ with\ Tensorflow/HorseHumanCNNClassification/horse-or-human.zip --id 1onaG42NZft3wCE1WH0GDEbUhu75fedP5 

Downloading...
From: https://drive.google.com/uc?id=1onaG42NZft3wCE1WH0GDEbUhu75fedP5
To: /content/drive/MyDrive/AI Basics with Tensorflow/HorseHumanCNNClassification/horse-or-human.zip
100% 150M/150M [00:00<00:00, 170MB/s]


In [4]:
# Download Horse/Human Validation Dataset to GDrive Designated Folder
!gdown --id 1LYeusSEIiZQpwN-mthh5nKdA75VsKG1U -O /content/drive/MyDrive/AI\ Basics\ with\ Tensorflow/HorseHumanCNNClassification/validation-horse-or-human.zip

Downloading...
From: https://drive.google.com/uc?id=1LYeusSEIiZQpwN-mthh5nKdA75VsKG1U
To: /content/drive/MyDrive/AI Basics with Tensorflow/HorseHumanCNNClassification/validation-horse-or-human.zip
100% 11.5M/11.5M [00:00<00:00, 101MB/s]


In [2]:
#Import TensorFloaw and Necessary Dependancies
import os
import zipfile
import tensorflow as tf

In [5]:
#Custom Callback Function to Terminate Training at 95% Accuracy
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if logs.get('accuracy') > 0.95:
      self.model.stop_training = True

In [6]:
# Unzip to Specific Location 

local_zip = os.path.join(path, 'horse-or-human.zip')
zip_ref = zipfile.ZipFile(local_zip, 'r')
training_folder = os.path.join(path,'horse-or-human')
zip_ref.extractall(training_folder)

local_zip = os.path.join(path, 'validation-horse-or-human.zip')
zip_ref = zipfile.ZipFile(local_zip, 'r')
validation_folder = os.path.join(path,'validation-horse-or-human')
zip_ref.extractall(validation_folder)

In [7]:
# Path to Training and Validation Dataset Folders and List of Horse/Human names

train_horse_dir = os.path.join(training_folder, 'horses')
train_human_dir = os.path.join(training_folder, 'humans')

validation_horse_dir = os.path.join(validation_folder, 'horses')
validation_human_dir = os.path.join(validation_folder, 'humans')

train_horse_names = os.listdir(train_horse_dir)
train_human_names = os.listdir(train_human_dir)

validation_horse_names = os.listdir(validation_horse_dir)
validation_human_names = os.listdir(validation_human_dir)


print(train_horse_names[:5])

['horse01-3.png', 'horse01-8.png', 'horse01-4.png', 'horse01-0.png', 'horse01-5.png']


In [10]:
#Create CNN Model

from tensorflow.keras.optimizers import RMSprop

model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape = (300, 300, 3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

#model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss= tf.keras.losses.BinaryCrossentropy, metrics='accuracy')
model.compile(optimizer=RMSprop(learning_rate=0.001), loss= 'BinaryCrossentropy', metrics='accuracy')

In [11]:
#Save Model Checkpoint to Google Drive During Training
checkpoint_path = "/content/drive/My Drive/AI Basics with Tensorflow/HorseHumanCNNClassification/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
# Custom Callback for Termination of Training at Desired Accuracy
callback = myCallback()

In [12]:
# Use ImageDataGenerator to Label Dataset Automatically Based Directory Structure
 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_gen = train_datagen.flow_from_directory(training_folder, target_size = (300,300), batch_size= 128, class_mode='binary')
validation_gen = validation_datagen.flow_from_directory(validation_folder, target_size= (300,300), batch_size=32, class_mode='binary')

history = model.fit(
      train_gen,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_gen,
      validation_steps = 8,
      callbacks = [cp_callback, callback]
      )

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - ETA: 0s - loss: 2.5329 - accuracy: 0.6263
Epoch 00001: saving model to /content/drive/My Drive/AI Basics with Tensorflow/HorseHumanCNNClassification/cp.ckpt
8/8 [==============================] - 12s 1s/step - loss: 2.5329 - accuracy: 0.6263 - val_loss: 3.7875 - val_accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - ETA: 0s - loss: 0.4612 - accuracy: 0.7553
Epoch 00002: saving model to /content/drive/My Drive/AI Basics with Tensorflow/HorseHumanCNNClassification/cp.ckpt
8/8 [==============================] - 9s 1s/step - loss: 0.4612 - accuracy: 0.7553 - val_loss: 1.4484 - val_accuracy: 0.6211
Epoch 3/15
8/8 [==============================] - ETA: 0s - loss: 0.5304 - accuracy: 0.8262
Epoch 00003: saving model to /content/drive/My Drive/AI Basics with Tensorflow/HorseHumanCNNClassification/cp.ckpt
8/8 [==============================] - 10s 

In [8]:
#Reload Model from GDrive Using Checkpoint and Evaluate to See the Same Result

from tensorflow.keras.optimizers import RMSprop

reloaded_model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape = (300, 300, 3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

reloaded_model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics='accuracy')

checkpoint_path = "/content/drive/My Drive/AI Basics with Tensorflow/HorseHumanCNNClassification/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

reloaded_model.load_weights(checkpoint_path)

#reloaded_model.evaluate(testing_images, testing_labels)

In [13]:
# To Test The Model With Unseen Dataset from Downloaded Internet

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = reloaded_model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving horse01-0.png to horse01-0.png
Saving horse1-000.png to horse1-000.png
Saving human01-00.png to human01-00.png
Saving valhuman01-00.png to valhuman01-00.png
[0.]
horse01-0.png is a horse
[0.]
horse1-000.png is a horse
[1.]
human01-00.png is a human
[1.]
valhuman01-00.png is a human
